<a href="https://colab.research.google.com/github/lucianosilvasp/qbitcoin/blob/main/mining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**CRYPTOCURRENCY MINING**

In [ ]:
!pip install qiskit-aer

In [ ]:
!pip install qiskit==0.46.1

**GROVER SEARCH WITH PROOF of WORK (PoW)**

Proof of work (PoW) describes a consensus mechanism that requires a significant amount of computing effort from a network of devices.

**KEY TAKEAWAYS**
* Proof of work (PoW) is a decentralized consensus mechanism that requires network members to expend effort in solving an encrypted hexadecimal number.
* Proof of work is also called **mining**, in reference to **receiving a reward** for work done.
* Proof of work allows for secure peer-to-peer transaction processing without needing a trusted third party.
* Proof of work at scale requires vast amounts of energy, which only increases as more miners join the network.

The concept was adapted digital tokens by Hal Finney in 2004 through the idea of "reusable proof of work" using the 160-bit secure hash algorithm 1 (SHA-1). Following its introduction in 2009, Bitcoin became the first widely adopted application of Finney's PoW idea (Finney was also the recipient of the first bitcoin transaction).

Commonly called a cryptocurrency, Bitcoin is technically a token—a representation of ownership of value on the Bitcoin blockchain. The ownership of the token can be exchanged for something of equal value, much like how you hand someone a dollar for a candy bar—they now have the dollar and you have the candy bar.

Blockchains are distributed ledgers that record all bitcoin transactions, similarly to how you would enter transactions in a spreadsheet. Each block is similar to a cell. Information such as transaction amounts, wallet addresses, time, and date are recorded and encrypted into a block header—a hexadecimal number created through the blockchain's **hashing function**.

The hash from each block is used in the block that follows it when its hash is created. This creates a ledger of chained blocks that cannot be altered because the information from every block is included in the newest block's hash.

**When a block is closed, the hash must be verified before a new block can be opened. This is where proof of work comes in.** The hash is a 64-digit encrypted hexadecimal number. With modern technology, a hash can be generated in milliseconds for a large amount of data. However, miners try to guess that hash, which takes a very long time in computing terms.

The hash includes a series of numbers called the nonce, short for "number used once." When a miner—the program on a node that works to solve the hash—begins mining, it generates a hash from publicly available information using a nonce equal to zero.

If the hash is lower than the current network target (or **accuracy**), the miner has successfully solved the hash. The network target is a mathematical result of a formula converted to a hexadecimal number that dictates the mining difficulty.

If the hash is greater than the target, the mining program adds a value of 1 to the nonce and generates a hash again. The entire network of miners tries to solve the hash this way.

On the Bitcoin blockchain, the miner that solves the hash is given the current reward for the work done.

In [14]:
from qiskit import *
#provider = IBMQ.enable_account('YOUR API KEY HERE')
#backend = provider.get_backend('ibm_oslo')

def applyHadamard(qc, qubits):
    for q in qubits:
        qc.h(q)

    return qc

def createBooleanOracle(input):
    oracle_circuit = QuantumCircuit(4,name="boolean oracle")
    for x in range(3):
        if input[x] == '0':
            oracle_circuit.x(x)

    oracle_circuit.mct([0,1,2], 3)

    for x in range(3):
        if input[x] == '0':
            oracle_circuit.x(x)

    return oracle_circuit.to_gate()

def createPhaseOracle(input):
    oracle_circuit = QuantumCircuit(3,name="phase oracle")
    for x in range(3):
        if input[x] == '0':
            oracle_circuit.x(x)

    oracle_circuit.ccz(0,1,2)

    for x in range(3):
        if input[x] == '0':
            oracle_circuit.x(x)

    return oracle_circuit.to_gate()

def amplificationGate():
    ampGate = QuantumCircuit(3,name="amplification gate")
    for x in range(3):
        ampGate.x(x)

    ampGate.ccz(0,1,2)

    for x in range(3):
        ampGate.x(x)

    for x in range(3):
        ampGate.h(x)

    return ampGate.to_gate()

def runSimulator(grover_circuit): # >70% accuracy  mine until 78% accuracy??????
    backend = Aer.get_backend('qasm_simulator')
    job = execute(grover_circuit, backend, shots=8192)
    result = job.result()
    counts = result.get_counts()
    return counts

def runReal(grover_circuit): # <50% accuracy
    job = execute(grover_circuit, backend, shots=8192)
    result = job.result()
    counts = result.get_counts()
    return counts

def mine(input, type_choice):
    grover_circuit = QuantumCircuit(4,3)
    grover_circuit.initialize('0000', grover_circuit.qubits)

    grover_circuit = applyHadamard(grover_circuit, [0,1,2,3])
    #grover_circuit.append(createBooleanOracle(input), [0,1,2,3])
    grover_circuit.append(createPhaseOracle(input), [0,1,2])
    grover_circuit = applyHadamard(grover_circuit, [0,1,2,3])
    grover_circuit.append(amplificationGate(), [0,1,2])
    grover_circuit.measure([0,1,2], [0,1,2])

    if type_choice == "1":
        counts = runSimulator(grover_circuit)
    if type_choice == "2":
        counts = runReal(grover_circuit)

    searchFor = input[::-1]
    accuracy = (counts[searchFor] / 8192) * 100
    return accuracy

The two most popular consensus mechanisms are proof of work and proof of stake. Bitcoin's top competitor Ethereum used proof of work on its blockchain until September 2022, when the highly-anticipated transition to proof of stake was made. Here are some of the key differences between the two.

**Proof of Work**

* Validation is done by a network of miners
* Bitcoin paid as a reward and for transaction fees
* Competitive nature uses lots of energy and computational power

**Proof of Stake**
* Validation is done by participants who offer ether as collateral
* Ether is paid for transaction fees only
* Less computational power and energy used

Proof of work **requires a computer to randomly engage** in hashing functions until it arrives at an output with the correct minimum amount of leading zeroes. For example, the hash for block #775,771, mined on Feb. 9, 2023, is:

00000000000000000003aa2696b1b7248db53a5a7f72d1fd98916c761e954354

There were 1,519 transactions in the block, and the total value of the block was 1,665.9645 BTC. This block was hashed by a miner 2.8 billion times (2.881.347.934) before reaching a number less than the target.

**The block reward for that successful hash was 6.25 BTC and 0.1360 BTC in fees. **

**BLOCKCHAIN SUPPORT AND HOW TO USE THE PROGRAM**

In [15]:
import time
from hashlib import sha256

class Block:
    blockHash = ''
    nonce = ''
    accuracy = 0

    def __init__(self, senderAddr, recieverAddr, amount, previousBlockHash, blockNumber):
        self.senderAddr = senderAddr
        self.recieverAddr = recieverAddr
        self.amount = amount
        self.previousBlockHash = previousBlockHash
        self.blockNumber = blockNumber
        self.timestamp = time.time()

        prevBlockHashBinary = ''.join(format(ord(i), '08b') for i in previousBlockHash)
        self.nonce = prevBlockHashBinary[-3:]
        counter = 0
        avgaccuracy = 0
        while(self.accuracy < chosen_accuracy):
            counter = counter + 1
            self.accuracy = mine(self.nonce, type_choice)
            avgaccuracy = avgaccuracy + self.accuracy
        print("Final Accuracy: ", self.accuracy)
        print("Iterations = ", counter)
        avgaccuracy = avgaccuracy / counter
        print("Average Accuracy = ", avgaccuracy)

        self.blockHash = previousBlockHash + senderAddr + recieverAddr + amount + str(self.timestamp) + str(self.nonce) + str(self.accuracy)
        self.blockHash = sha256(self.blockHash.encode('utf-8')).hexdigest()



class Blockchain:
    blockchain = []
    def __init__(self):
        if len(self.blockchain) < 1:
            genBlock = Block('Genesis', 'Genesis', '0', '0', 1)
            self.blockchain.append(genBlock)

    def addBlock(self, additionalBlock):
        self.blockchain.append(additionalBlock)

    def printChain(self):
        print('Block : Blockhash                                                         : Reciever : Sender : Amount : Accuracy')
        for x in range(len(self.blockchain)):
            print(self.blockchain[x].blockNumber, self.blockchain[x].blockHash, ' : ', self.blockchain[x].recieverAddr, ' : ', self.blockchain[x].senderAddr, ' : ', self.blockchain[x].amount, ' : ',self.blockchain[x].accuracy)

def userChoice1(blockchain):

    print("Please input sender address: ")
    sendrAddr = input()
    print("Please input reciever address: ")
    recvAddr = input()
    print("Enter amount: ")
    amount = input()

    chainLength = len(blockchain.blockchain)
    previousBlockNumber = blockchain.blockchain[chainLength - 1].blockNumber
    prevBlockHash = blockchain.blockchain[chainLength - 1].blockHash

    currentBlock = Block(sendrAddr, recvAddr, amount, prevBlockHash, previousBlockNumber + 1)
    blockchain.addBlock(currentBlock)

def main():
    global sim_accuracy
    sim_accuracy = 78.8
    global quantum_accuracy
    quantum_accuracy = 45.5
    global type_choice

    print("Please choose (1) simulator or (2) real quantum version: ")
    type_choice = input()
    match type_choice:
        case "1":
            global chosen_accuracy
            chosen_accuracy = sim_accuracy
        case "2":
            chosen_accuracy = quantum_accuracy

    blockchain = Blockchain()

    while(1):
        print("Choose option: ")
        print("1. Add new block to chain.")
        print("2. Display blocks on current blockchain.")
        choice = input()

        match choice:
            case "1":
                userChoice1(blockchain)
            case "2":
                blockchain.printChain()
            case _:
                print("Invalid choice")

In [ ]:
main()